In [ ]:
# UCI Breast Cancer Dataset - SVM Model Training

## Table of Contents
1. Load Preprocessed Data
2. Train SVM Model
3. Evaluate Performance
4. Confusion Matrix & Classification Report
5. ROC Curve Analysis
6. Hyperparameter Tuning with Grid Search
7. Save Best Model
8. Make Predictions

In [ ]:
## 1. Import  Libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from time import time

# SVM and model selection
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix, 
    classification_report,
    roc_auc_score,
    roc_curve
)

import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("imported successfully")


In [ ]:
## 2. Loading previously preprocessed Data


In [ ]:
# Load preprocessed data
print("Loading preprocessed data...")

X_train = pd.read_csv('data/processed/X_train_scaled.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
X_test = pd.read_csv('data/processed/X_test_scaled.csv')
y_test = pd.read_csv('data/processed/y_test.csv')

# Convert to numpy arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

print(f"✓ Data loaded successfully!")
print(f"\nDataset Information:")
print(f"  Training samples: {X_train.shape[0]}")
print(f"  Testing samples: {X_test.shape[0]}")
print(f"  Number of features: {X_train.shape[1]}")
print(f"\nClass Distribution:")
print(f"  Training - Benign: {np.sum(y_train == 0)}, Malignant: {np.sum(y_train == 1)}")
print(f"  Testing - Benign: {np.sum(y_test == 0)}, Malignant: {np.sum(y_test == 1)}")


In [ ]:
## 3. Train Basic SVM Model
basic svm with rbf

In [ ]:
# Create and train SVM model
print("Training SVM model with RBF kernel...")
print("="*60)

start_time = time()

# Initialize SVM with RBF kernel
svm_model = SVC(kernel='rbf', random_state=42, probability=True)

# Train the model
svm_model.fit(X_train, y_train)

training_time = time() - start_time

print(f"Model trained successfully in {training_time:.2f} seconds!")
print(f"\nModel Parameters:")
print(f"  Kernel: {svm_model.kernel}")
print(f"  C (regularization): {svm_model.C}")
print(f"  Gamma: {svm_model.gamma}")


In [ ]:
## 4. Make Predictions and Evaluate Performance

In [ ]:
# Make predictions
y_pred = svm_model.predict(X_test)
y_pred_proba = svm_model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n" + "="*60)
print("SVM MODEL PERFORMANCE")
print("="*60)
print(f"\nAccuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
print(f"F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
print(f"ROC-AUC:   {roc_auc:.4f} ({roc_auc*100:.2f}%)")
print("="*60)

# Cross-validation score
cv_scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"\n5-Fold Cross-Validation Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

In [ ]:
## 5. Confusion Matrix Visualization

In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Benign (0)', 'Malignant (1)'],
            yticklabels=['Benign (0)', 'Malignant (1)'],
            annot_kws={'size': 16, 'weight': 'bold'})
plt.title('Confusion Matrix - SVM Model', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)

# Add text annotations
tn, fp, fn, tp = cm.ravel()
plt.text(0.5, -0.15, f'True Negatives: {tn}', ha='center', transform=plt.gca().transAxes, fontsize=10)
plt.text(0.5, -0.20, f'False Positives: {fp}', ha='center', transform=plt.gca().transAxes, fontsize=10)
plt.text(0.5, -0.25, f'False Negatives: {fn}', ha='center', transform=plt.gca().transAxes, fontsize=10)
plt.text(0.5, -0.30, f'True Positives: {tp}', ha='center', transform=plt.gca().transAxes, fontsize=10)

plt.tight_layout()
plt.show()

print("\nConfusion Matrix Breakdown:")
print(f"  True Negatives (Correct Benign):    {tn}")
print(f"  False Positives (Benign as Malignant): {fp}")
print(f"  False Negatives (Malignant as Benign): {fn}")
print(f"  True Positives (Correct Malignant):  {tp}")


In [ ]:
## 6. Detailed Classification Report

In [ ]:
# Print detailed classification report
print("\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_pred, 
                          target_names=['Benign (0)', 'Malignant (1)'],
                          digits=4))
print("="*60)

In [ ]:
## 7. ROC Curve Analysis

In [ ]:
# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='#e74c3c', linewidth=3, label=f'SVM (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier (AUC = 0.5000)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('ROC Curve - SVM Model', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nROC-AUC Score: {roc_auc:.4f}")
print(f"  This means the model has a {roc_auc*100:.2f}% chance of correctly distinguishing")
print(f"  between benign and malignant cases.")